In [1]:
import pandas as pd
import numpy as np

In [2]:
from Bio.Seq import Seq
from Bio import SeqIO
#from Bio.Alphabet import IUPAC
import sys
# import pyensembl
import os
#https://towardsdatascience.com/a-simple-guide-to-command-line-arguments-with-argparse-6824c30ab1c3


In [4]:
# SK: Dictionary of protein names to sequences
proteins = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_translations.fa", "fasta"):
    name = record.id.split("|")[1].split(".")[0]
    proteins[name] = str(record.seq)
    

# SK: Dictionary of protein names to CDS dna transcript

dna_transcripts = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_transcripts.fa", "fasta"):
	name = record.id.split("|")[0].split(".")[0]
	record_c = record.id.split("|")
	for i in record_c:
		if "CDS" in i:
			coords = i.replace("CDS:","")
	start = int(coords.split("-")[0])
	end = int(coords.split("-")[1])
	dna_seq = str(record.seq)[start-1:end]
	dna_transcripts[name] = dna_seq

# Plan:
1. Use cds coordinates, create df with genomic coordinates as one column and corresponding nt as second column
2. Use position in variant file to resplace the value in the nt column
3. Translate the nt column

In [8]:
uniprot_id = "P32242"
ENST = "ENST00000282549"

In [9]:
variants = pd.read_csv("../outputs/mutations/domains_expanded_iWES_v2_variants_snv_classified/" + ENST + ".bed", sep = "\t", header = None)
no_syn_variants = variants[variants[22] == "No-Syn"]
no_syn_variants

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
2,2,63055786,63055787,AD,ENSG00000115507,.,+,ENST00000282549,2,63055786,...,+,2,63055786,63055787,C,T,0.000005,P,L,No-Syn
4,2,63055789,63055790,AD,ENSG00000115507,.,+,ENST00000282549,2,63055789,...,+,2,63055789,63055790,C,T,0.000009,A,V,No-Syn
5,2,63055791,63055792,AD,ENSG00000115507,.,+,ENST00000282549,2,63055791,...,+,2,63055791,63055792,T,C,0.000014,S,P,No-Syn
7,2,63055800,63055801,AD,ENSG00000115507,.,+,ENST00000282549,2,63055800,...,+,2,63055800,63055801,C,G,0.000009,P,A,No-Syn
8,2,63055800,63055801,AD,ENSG00000115507,.,+,ENST00000282549,2,63055800,...,+,2,63055800,63055801,C,T,0.000005,P,S,No-Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2,63056263,63056264,AD,ENSG00000115507,.,+,ENST00000282549,2,63056263,...,+,2,63056263,63056264,C,A,0.000009,P,H,No-Syn
127,2,63056269,63056270,AD,ENSG00000115507,.,+,ENST00000282549,2,63056269,...,+,2,63056269,63056270,G,C,0.000023,C,S,No-Syn
129,2,63056282,63056283,AD,ENSG00000115507,.,+,ENST00000282549,2,63056282,...,+,2,63056282,63056283,G,T,0.000005,K,N,No-Syn
130,2,63056289,63056290,AD,ENSG00000115507,.,+,ENST00000282549,2,63056289,...,+,2,63056289,63056290,G,T,0.000005,A,S,No-Syn


In [10]:
strand = no_syn_variants[13].iloc[1]
strand

'+'

In [11]:
wt_nt = dna_transcripts[ENST]
wt_nt

'ATGATGTCTTACCTCAAACAACCCCCATACGGCATGAACGGGCTGGGCCTGGCCGGGCCCGCCATGGACCTCCTGCACCCATCCGTGGGCTATCCGGCCACTCCGCGGAAGCAGCGGCGGGAGCGCACCACCTTCACGCGTTCACAGCTGGACGTGCTCGAGGCGCTCTTCGCCAAGACTCGCTACCCTGACATCTTCATGCGGGAGGAGGTGGCGCTCAAGATCAACCTGCCGGAGTCTAGAGTCCAGGTCTGGTTCAAGAACCGCCGCGCCAAATGCCGCCAGCAGCAGCAGAGCGGGAGCGGAACCAAGAGCCGCCCAGCCAAGAAGAAGTCCTCTCCAGTGCGGGAGAGCTCGGGCTCCGAAAGCAGTGGCCAATTCACGCCGCCAGCTGTGTCCAGCTCTGCCTCGTCCTCTAGCTCGGCGTCCAGCTCTTCCGCCAACCCAGCGGCTGCAGCGGCTGCGGGACTAGGTGGGAACCCGGTGGCGGCCGCGTCGTCGCTGAGTACACCAGCTGCCTCATCTATCTGGAGCCCGGCCTCCATCTCGCCAGGCTCAGCGCCCGCGTCCGTGTCGGTGCCGGAGCCATTGGCCGCGCCTAGCAACACCTCGTGTATGCAGCGCTCCGTAGCTGCAGGCGCCGCCACCGCAGCAGCCTCTTATCCCATGTCCTACGGCCAGGGCGGCAGCTACGGCCAAGGCTACCCTACGCCCTCCTCTTCCTACTTTGGCGGCGTGGACTGCAGCTCATACCTAGCGCCCATGCACTCACATCACCACCCGCACCAGCTCAGCCCCATGGCACCCTCCTCCATGGCGGGCCACCATCATCACCACCCACATGCGCACCACCCGTTGAGCCAGTCCTCAGGCCACCACCACCACCATCACCACCACCACCACCAAGGCTACGGTGGCTCTGGGCTTGCCTTCAACTCTGCCGACTGCTTGGATTACAAGGAGCCTGGCGCCGCTGCTGCTTCCTCCGCCTGGAAACTC

In [12]:
if strand == "-":
    wt_nt = str(Seq(wt_nt).reverse_complement())
wt_nt

'ATGATGTCTTACCTCAAACAACCCCCATACGGCATGAACGGGCTGGGCCTGGCCGGGCCCGCCATGGACCTCCTGCACCCATCCGTGGGCTATCCGGCCACTCCGCGGAAGCAGCGGCGGGAGCGCACCACCTTCACGCGTTCACAGCTGGACGTGCTCGAGGCGCTCTTCGCCAAGACTCGCTACCCTGACATCTTCATGCGGGAGGAGGTGGCGCTCAAGATCAACCTGCCGGAGTCTAGAGTCCAGGTCTGGTTCAAGAACCGCCGCGCCAAATGCCGCCAGCAGCAGCAGAGCGGGAGCGGAACCAAGAGCCGCCCAGCCAAGAAGAAGTCCTCTCCAGTGCGGGAGAGCTCGGGCTCCGAAAGCAGTGGCCAATTCACGCCGCCAGCTGTGTCCAGCTCTGCCTCGTCCTCTAGCTCGGCGTCCAGCTCTTCCGCCAACCCAGCGGCTGCAGCGGCTGCGGGACTAGGTGGGAACCCGGTGGCGGCCGCGTCGTCGCTGAGTACACCAGCTGCCTCATCTATCTGGAGCCCGGCCTCCATCTCGCCAGGCTCAGCGCCCGCGTCCGTGTCGGTGCCGGAGCCATTGGCCGCGCCTAGCAACACCTCGTGTATGCAGCGCTCCGTAGCTGCAGGCGCCGCCACCGCAGCAGCCTCTTATCCCATGTCCTACGGCCAGGGCGGCAGCTACGGCCAAGGCTACCCTACGCCCTCCTCTTCCTACTTTGGCGGCGTGGACTGCAGCTCATACCTAGCGCCCATGCACTCACATCACCACCCGCACCAGCTCAGCCCCATGGCACCCTCCTCCATGGCGGGCCACCATCATCACCACCCACATGCGCACCACCCGTTGAGCCAGTCCTCAGGCCACCACCACCACCATCACCACCACCACCACCAAGGCTACGGTGGCTCTGGGCTTGCCTTCAACTCTGCCGACTGCTTGGATTACAAGGAGCCTGGCGCCGCTGCTGCTTCCTCCGCCTGGAAACTC

In [13]:
nt_df = pd.DataFrame({"nt" : [*wt_nt]})
nt_df

,nt
0,A
1,T
2,G
3,A
4,T
...,...
1060,T
1061,G
1062,T
1063,G


In [16]:
cds_bed = pd.read_csv("../outputs/mutations/cds_bed_format/" + ENST, sep = "\t", header = None)
cds_bed = cds_bed[[1, 2]]
cds_bed[1] += 1
cds_bed

,1,2
0,63052991,63053087
1,63054047,63054198
2,63055501,63056316


In [17]:
# cds_bed = cds_bed.sort_values(by = 1, ascending = False)
# cds_bed

In [18]:
range_col = []
for start, end in zip(cds_bed[1], cds_bed[2]):
    if strand == "-":
        range_col += list(range(end, start - 1, -1))
    else:
        range_col += list(range(start, end + 1))

In [19]:
len(range_col)

1065

In [20]:
nt_df["gen_pos"] = range_col
nt_df = nt_df.set_index("gen_pos")
nt_df

,nt
gen_pos,
63052991,A
63052992,T
63052993,G
63052994,A
63052995,T
...,...
63056312,T
63056313,G
63056314,T


In [22]:
nt_df.loc[63055787]

nt    C
Name: 63055787, dtype: object

In [23]:
nt_df.loc[63055790]

nt    C
Name: 63055790, dtype: object

In [25]:
nt_df.loc[63055792]

nt    T
Name: 63055792, dtype: object

In [21]:
no_syn_variants

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
2,2,63055786,63055787,AD,ENSG00000115507,.,+,ENST00000282549,2,63055786,...,+,2,63055786,63055787,C,T,0.000005,P,L,No-Syn
4,2,63055789,63055790,AD,ENSG00000115507,.,+,ENST00000282549,2,63055789,...,+,2,63055789,63055790,C,T,0.000009,A,V,No-Syn
5,2,63055791,63055792,AD,ENSG00000115507,.,+,ENST00000282549,2,63055791,...,+,2,63055791,63055792,T,C,0.000014,S,P,No-Syn
7,2,63055800,63055801,AD,ENSG00000115507,.,+,ENST00000282549,2,63055800,...,+,2,63055800,63055801,C,G,0.000009,P,A,No-Syn
8,2,63055800,63055801,AD,ENSG00000115507,.,+,ENST00000282549,2,63055800,...,+,2,63055800,63055801,C,T,0.000005,P,S,No-Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2,63056263,63056264,AD,ENSG00000115507,.,+,ENST00000282549,2,63056263,...,+,2,63056263,63056264,C,A,0.000009,P,H,No-Syn
127,2,63056269,63056270,AD,ENSG00000115507,.,+,ENST00000282549,2,63056269,...,+,2,63056269,63056270,G,C,0.000023,C,S,No-Syn
129,2,63056282,63056283,AD,ENSG00000115507,.,+,ENST00000282549,2,63056282,...,+,2,63056282,63056283,G,T,0.000005,K,N,No-Syn
130,2,63056289,63056290,AD,ENSG00000115507,.,+,ENST00000282549,2,63056289,...,+,2,63056289,63056290,G,T,0.000005,A,S,No-Syn


In [31]:
for i in no_syn_variants.index:
    pos = no_syn_variants[2].loc[i]
    sfari_nt = no_syn_variants[no_syn_variants[2] == pos][17].loc[i]
    nt_df_nt = nt_df.loc[pos].iloc[0]
    if sfari_nt != nt_df_nt:
        print("mismatch!")

Everything seems to work on the positive strand!